# Quick-start

This document helps you get up-and-running with `alr` immediately.
It should give you a general idea of how to get started with
this package.


In [ ]:
import numpy as np
import torch
import torch.utils.data as torchdata

from torch.nn import functional as F
from torch import nn

from alr import MCDropout
from alr.acquisition import RandomAcquisition
from alr.utils import stratified_partition
from alr.data import DataManager, UnlabelledDataset
from alr.data.datasets import Dataset

np.random.seed(42)
torch.manual_seed(42)
data_loader_params = dict(pin_memory=True, num_workers=2, batch_size=32)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Firstly, we load and prepare our data.
Note that we partitioned the training set into labelled and unlabelled sets
using `stratified partition` which balances the number of classes in the training pool:

In [ ]:
# load training data
train, test = Dataset.MNIST.get()
train, pool = stratified_partition(train, Dataset.MNIST.about.n_class, 20)
pool = UnlabelledDataset(pool)
len(train), len(test), len(pool)

`MCDropout` lets us define a Bayesian NN. It provides an implementation
for `stochastic_forward` which we will use in the next section for the
acquisition function.

> Notice the dropout layers have been changed to their `Persistent` versions.

In [ ]:
# instantiate a regular model and an optimiser.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 1024)
        self.drop1 = nn.Dropout()
        self.fc2 = nn.Linear(1024, 2048)
        self.drop2 = nn.Dropout()
        self.fc3 = nn.Linear(2048, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.drop1(self.fc1(x)))
        x = F.relu(self.drop2(self.fc2(x)))
        return self.fc3(x)

model = MCDropout(Net(), apply_logsoft=True, forward=10).to(device)
model.compile(criterion=torch.nn.NLLLoss(),  # since apply_logsoft=True
              optimiser=torch.optim.Adam(model.parameters()))
model

Now, we can instantiate an acquisition function
and an associated `DataManager` instance:

In [ ]:
ra = RandomAcquisition()
dm = DataManager(train, pool, ra)

Finally, the vanilla acquisition loop looks like:

In [ ]:
ITERS = 5
EPOCHS = 5
accs = {}

# In each iteration, acquire `b` points
for i in range(ITERS):
    print(f"==== Iteration {i + 1} ({(i + 1) / ITERS:.2%}), "
          f"training size: {dm.n_labelled} ====")
    # reset weights to original values when the model was first created
    model.reset_weights()
    # fit = train
    result = model.fit(train_loader=torchdata.DataLoader(dm.labelled, **data_loader_params),
                       train_acc=True, epochs=EPOCHS, device=device)
    # evaluate the model to obtain its test accuracy
    test_acc, _ = model.evaluate(data=torchdata.DataLoader(test, **data_loader_params), device=device)
    # display results
    result.reduce('last', inplace=True)
    accs[dm.n_labelled] = test_acc
    print(f"train_acc = {result.train_acc:.2f}, "
          f"train_loss = {result.train_loss:.2f}, "
          f"test_acc = {test_acc:.2f}")
    # acquire `b` points from unlabelled pool
    dm.acquire(b=10)
accs
